# A Priori

> Auteur : Mathilde Boltenhagen, dans un projet en trinôme avec Isabelle Flores et Alice Montel

> Date : Janvier 2018 

L'idée de ce notebook est d'extraire les règles d'association à l'aide du package R "arules". Le dataset représente des données de goût musicaux et des critères démographiques. 


Nous commençons tout d'abord par charger les données et regardons un aperçu. 

In [2]:
# chargement des données
data <- read.table('responses.csv',header=TRUE,sep=",",stringsAsFactors = TRUE)
# affichage de la taille de la matrice
print(dim(data))
# affichage des 5 premières lignes
print(head(data, 5))

[1] 1010  150
  Music Slow.songs.or.fast.songs Dance Folk Country Classical.music Musical Pop
1     5                        3     2    1       2               2       1   5
2     4                        4     2    1       1               1       2   3
3     5                        5     2    2       3               4       5   3
4     5                        3     2    1       1               1       1   2
5     5                        3     4    3       2               4       3   5
  Rock Metal.or.Hardrock Punk Hiphop..Rap Reggae..Ska Swing..Jazz Rock.n.roll
1    5                 1    1           1           1           1           3
2    5                 4    4           1           3           1           4
3    5                 3    4           1           4           3           5
4    2                 1    4           2           2           1           2
5    3                 1    2           5           3           2           1
  Alternative Latino Techno..Trance Op

Nous remarquons qu'il y a des données catégorielles. Celles-ci vont être transformées en m colonnes binaires à l'aide de la librairie Dummies. 

In [3]:
library(dummies)
# ajout de m colonnes avec _ pour séparer le nom de la colonne et le choix 
data_new = dummy.data.frame(data, sep = "_")
print(names(data_new))

dummies-1.5.6 provided by Decision Patterns



  [1] "Music"                                     
  [2] "Slow.songs.or.fast.songs"                  
  [3] "Dance"                                     
  [4] "Folk"                                      
  [5] "Country"                                   
  [6] "Classical.music"                           
  [7] "Musical"                                   
  [8] "Pop"                                       
  [9] "Rock"                                      
 [10] "Metal.or.Hardrock"                         
 [11] "Punk"                                      
 [12] "Hiphop..Rap"                               
 [13] "Reggae..Ska"                               
 [14] "Swing..Jazz"                               
 [15] "Rock.n.roll"                               
 [16] "Alternative"                               
 [17] "Latino"                                    
 [18] "Techno..Trance"                            
 [19] "Opera"                                     
 [20] "Movies"                 

La plupart des variables ont une notation entre 1 et 5 ce qui compliquent beaucoup les règles d'associations. Nous supposons qu'une personne valide une réponse si elle répond positivement c'est à dire supérieur à la médiane des choix possibles. Par exemple lorsqu'il y a une notation entre 1 et 5, les seules notes 4 et 5 sont comptés comme positives.

In [4]:
# Pour chaque variables, mettre à 1 les personnes ayant répondu positivement
for(column in names(data_new)){
  values <-levels(as.factor((data_new[,column])))
  # la valeur de réponse doit être supérieur à la médiane des réponses possible 
  like <-which(data_new[,column]> median(as.numeric(values),na.rm = TRUE)  )
  data_new[like,column]=1 # la personne répond positivement
  
  dislike<-which(data_new[,column]!=1)
  data_new[dislike,column]=0 # la personne répond négativement
}

Nous allons maintenant utiliser le package "arules". 

In [5]:
library("arules")
# les données sont transformées en facteurs afin d'avoir plus d'options
data_factors= as.data.frame(lapply(data_new, as.factor))

Warning message:
"package 'arules' was built under R version 3.4.3"Loading required package: Matrix

Attaching package: 'arules'

The following objects are masked from 'package:base':

    abbreviate, write



On sépare les données démographiques et nos données sur les goûts musicaux qui nous intéressent ici. 

In [6]:
# données démographiques
data_demog=data_factors[20:ncol(data_factors)]

# musiques
data_music= data_factors[3:19]

On peut alors créer les règles d'association en utilisant le package arules. Ici nous avons choisi un support minimal de 10% ce qui est assez élevé. La confiance d'une règle vaudra 60%. On oblige la règle à valoir entre 2 et 4 termes. 

Nous cherchons spécialement les règles qui impliquent un style de musique afin de deviner le style de musique d'une personne selon sa personnalité. 

In [ ]:
# pour chaque style de musique
for(name in names(data_music)){
  # on cherche les règles où la conclusion est "aime le style"
  condition=paste(name, "=",1, sep="")
  # sink() permet de supprimer des logs de arules qui sont très nombreux
  sink(tempfile())
  rule= apriori(data.frame(data_demog,data_factors[name]), parameter = list(support=0.10 , confidence=0.6, minlen=2, maxlen=4,  ext=T  ),
                appearance = list (rhs=condition, default="lhs"))
  sink()
  # on ouvre un sink pour les résultats
  sink(file = "./result.txt",append = TRUE)
  # on prend les premiers, triés par confiance
  inspect(head(sort(rule, by="confidence"),10))
  sink()
}

#system("cat result.txt")
